# Comparativa de Modelos de Regresión: Temperatura y Humedad

Se comparan los principales modelos (Lineal, Polinómica, Ridge, Lasso, ElasticNet, Kernel Ridge) para ambos targets: Temperatura y Humedad. Se muestra una tabla resumen de R² para cada modelo y variable objetivo.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import r2_score

In [ ]:
# Cargar datos
data = pd.read_csv('../dataset/ML1_temperature.xlsx - Hoja1.csv', decimal=',')
X = data.drop(['recnt_Humidity','recnt_Temperature'], axis=1).values
y_temp = data['recnt_Temperature'].values.reshape(-1, 1)
y_hum = data['recnt_Humidity'].values.reshape(-1, 1)

In [ ]:
def comparar_modelos(X, y, nombre_target):
    resultados = {}
    # Escalado
    scaler_x = StandardScaler()
    scaler_y = StandardScaler()
    Xs = scaler_x.fit_transform(X)
    ys = scaler_y.fit_transform(y)
    x_train, x_val, y_train, y_val = train_test_split(Xs, ys, test_size=0.2, random_state=42)
    # Lineal
    lr = LinearRegression().fit(x_train, y_train)
    y_pred = lr.predict(x_val)
    resultados['Lineal'] = r2_score(y_val, y_pred)
    # Polinómica (mejor grado 1-5)
    best_r2 = -np.inf; best_deg = 1
    for d in range(1,6):
        poly = PolynomialFeatures(degree=d)
        xtr = poly.fit_transform(x_train)
        xva = poly.transform(x_val)
        lr_poly = LinearRegression().fit(xtr, y_train)
        y_pred_poly = lr_poly.predict(xva)
        r2 = r2_score(y_val, y_pred_poly)
        if r2 > best_r2: best_r2 = r2; best_deg = d
    resultados[f'Polinómica (grado {best_deg})'] = best_r2
    # Ridge
    ridge = GridSearchCV(Ridge(), {'alpha': np.logspace(-3,2,10)}, cv=5, scoring='r2').fit(x_train, y_train.ravel())
    resultados['Ridge'] = ridge.best_score_
    # Lasso
    lasso = GridSearchCV(Lasso(max_iter=10000), {'alpha': np.logspace(-3,2,10)}, cv=5, scoring='r2').fit(x_train, y_train.ravel())
    resultados['Lasso'] = lasso.best_score_
    # ElasticNet
    enet = GridSearchCV(ElasticNet(max_iter=10000), {'alpha': np.logspace(-3,2,10), 'l1_ratio':[0.1,0.5,0.9]}, cv=5, scoring='r2').fit(x_train, y_train.ravel())
    resultados['ElasticNet'] = enet.best_score_
    # Kernel Ridge (RBF, búsqueda de gamma)
    best_r2_krr = -np.inf; best_gamma = 0.1
    for gamma in [0.001, 0.01, 0.1, 1, 10]:
        krr = KernelRidge(kernel='rbf', gamma=gamma)
        krr.fit(x_train, y_train)
        y_pred_krr = krr.predict(x_val)
        r2_krr = r2_score(y_val, y_pred_krr)
        if r2_krr > best_r2_krr: best_r2_krr = r2_krr; best_gamma = gamma
    resultados[f'Kernel Ridge (gamma={best_gamma})'] = best_r2_krr
    return resultados

In [ ]:
res_temp = comparar_modelos(X, y_temp, 'Temperatura')
res_hum = comparar_modelos(X, y_hum, 'Humedad')
tabla = pd.DataFrame({'Temperatura': res_temp, 'Humedad': res_hum})
tabla = tabla.T
display(tabla.style.format('{:.3f}').set_caption('R² de cada modelo para Temperatura y Humedad'))

**Conclusión:**

Revisa la tabla para identificar el modelo con mayor R² para cada variable. Así puedes elegir el más adecuado para temperatura y para humedad.